In [1]:
from dotenv import load_dotenv
import os

# 1st method: using .env file.
load_dotenv()
# Access them using os.getenv or os.environ
api_key = os.getenv("GROQ_API_KEY")

# 2nd method: using hard code
# api_key = "<put the api key here>"
# if not os.environ.get("GROQ_API_KEY"):
#     os.environ["GROQ_API_KEY"] = api_key #getpass.getpass("Enter API key for Groq: ")





In [ ]:

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

## LLMChain vs SequentialChain

In [2]:
# to be filled ...

##   llm.run() vs and llm.invoke()

In [3]:
# to be filled...

## ChatPromptTemplate vs PromptTemplate

* What is **PromptTemplate**?

    A template for traditional completion models, like text-davinci-003 or OpenAI() in text mode.
    - used when:
    You’re using text completion models
    You want a simple string prompt





* What is **ChatPromptTemplate**?

    A template for chat models, like gpt-3.5-turbo or gpt-4, which expect structured message history (system, user, assistant messages).



    - used when:
    You’re using chat-based models
    You want to structure a conversation with multiple roles


In [4]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is the capital of {country}?")
prompt.format(country="Japan")
# Output: "What is the capital of Japan?"

'What is the capital of Japan?'

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "What is the capital of {country}?")
])
prompt.format_messages(country="Japan")
# Output: list of ChatMessages (SystemMessage, HumanMessage)


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the capital of Japan?', additional_kwargs={}, response_metadata={})]

| Feature                         | `PromptTemplate`                        | `ChatPromptTemplate`                             |
|----------------------------------|-----------------------------------------|--------------------------------------------------|
| Model Type                      | Text completion (e.g., `text-davinci-003`) | Chat models (e.g., `gpt-3.5-turbo`)            |
| Output                          | Single formatted string                 | List of structured `ChatMessage` objects         |
| Multi-turn interactions         | ❌ No (flat prompt)                     | ✅ Yes (role-based message turns)                |
| Message roles (`system`, `user`, `assistant`) | ❌ No                          | ✅ Yes                                           |


In [6]:
# so what are completion or chat models and their differences?

| Feature                          | Completion Models                          | Chat Models                                     |
|----------------------------------|---------------------------------------------|--------------------------------------------------|
| Input Format                     | Plain text string                          | List of messages with roles (system/user/etc.)  |
| Designed for                     | One-shot or few-shot tasks                 | Conversational interactions                     |
| Model Examples                   | `text-davinci-003`, `GPT-Neo`, `GPT-J`     | `gpt-3.5-turbo`, `gpt-4`, `Claude`, `Gemini`    |
| Supports roles (user/system)     | ❌ No                                       | ✅ Yes                                           |
| Output Style                     | Text continuation                          | Role-based response from assistant              |
| Memory-aware                     | ❌ Not naturally                            | ✅ Yes (chat history maintained)                 |
| Performance (for chat tasks)     | ⚠️ Often worse                             | ✅ Optimized for dialogue                        |
| Needs manual formatting          | ✅ Yes                                      | ❌ No                                            |
| Hard to manage multi-turn input  | ✅ Yes                                      | ❌ No                                            |
| Best for                         | Simple or isolated tasks                   | Interactive, multi-turn conversations           |


In [7]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])

memory = ConversationBufferMemory(return_messages=True)


chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

response = chain.run(input="What's the capital of France?")
# Automatically stores and injects conversation history



response

/var/folders/1k/dtgfp49j1f92ysb1xwlxsnsm0000gp/T/ipykernel_42051/1803211992.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
/var/folders/1k/dtgfp49j1f92ysb1xwlxsnsm0000gp/T/ipykernel_42051/1803211992.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
/var/folders/1k/dtgfp49j1f92ysb1xwlxsnsm0000gp/T/ipykernel_42051/1803211992.py:20: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(input="What's the capital of France?")


"I'd be happy to help! According to our chat history, this is the first message you've sent. Therefore, I'll provide the answer: The capital of France is Paris."

In [8]:
response = chain.run(input="What's the population of that?")


response

"According to our chat history, you've asked about the capital of France, which is Paris. As for the population, Paris has a population of approximately 2.1 million people within its city limits. However, the larger metropolitan area of Paris has a population of over 12.2 million people."

In [9]:
# now the question is how to see the moemory?

In [10]:
for msg in memory.chat_memory.messages:
    print(f"{msg.type.capitalize()}: {msg.content}")

Human: What's the capital of France?
Ai: I'd be happy to help! According to our chat history, this is the first message you've sent. Therefore, I'll provide the answer: The capital of France is Paris.
Human: What's the population of that?
Ai: According to our chat history, you've asked about the capital of France, which is Paris. As for the population, Paris has a population of approximately 2.1 million people within its city limits. However, the larger metropolitan area of Paris has a population of over 12.2 million people.


In [4]:
# you can see that both turns in messages are stored in mem now

In [11]:
response = chain.run(input="the most famouse person ever lived there?")
response = chain.run(input="is is larger than London (by land)?")


In [12]:

for msg in memory.chat_memory.messages:
    print(f"{msg.type.capitalize()}: {msg.content}")

Human: What's the capital of France?
Ai: I'd be happy to help! According to our chat history, this is the first message you've sent. Therefore, I'll provide the answer: The capital of France is Paris.
Human: What's the population of that?
Ai: According to our chat history, you've asked about the capital of France, which is Paris. As for the population, Paris has a population of approximately 2.1 million people within its city limits. However, the larger metropolitan area of Paris has a population of over 12.2 million people.
Human: the most famouse person ever lived there?
Ai: According to our chat history, you've asked about the capital of France, which is Paris, and also about the population of Paris. As for your latest question, one of the most famous people to have lived in Paris is Vincent van Gogh, a Dutch post-impressionist artist who spent time in the city and created many of his famous works there.
Human: is is larger than London (by land)?
Ai: According to our chat history, y

In [13]:
# how many messages so far are there in mem?
print(len(memory.chat_memory.messages))

8


In [14]:
# # To clear memory: (uncomment if you want this)
# memory.clear()


In [15]:
# Save messages manually:
from langchain.schema import HumanMessage, AIMessage
memory.chat_memory.add_user_message("What is the best place there to visit?")
memory.chat_memory.add_ai_message("House of Meoosiso")

In [16]:
for msg in memory.chat_memory.messages:
    print(f"{msg.type.capitalize()}: {msg.content}")

Human: What's the capital of France?
Ai: I'd be happy to help! According to our chat history, this is the first message you've sent. Therefore, I'll provide the answer: The capital of France is Paris.
Human: What's the population of that?
Ai: According to our chat history, you've asked about the capital of France, which is Paris. As for the population, Paris has a population of approximately 2.1 million people within its city limits. However, the larger metropolitan area of Paris has a population of over 12.2 million people.
Human: the most famouse person ever lived there?
Ai: According to our chat history, you've asked about the capital of France, which is Paris, and also about the population of Paris. As for your latest question, one of the most famous people to have lived in Paris is Vincent van Gogh, a Dutch post-impressionist artist who spent time in the city and created many of his famous works there.
Human: is is larger than London (by land)?
Ai: According to our chat history, y

In [17]:


response = chain.run(input="What is the best place there to visit?")
response

'I apologize for the mistake earlier! As for your question, the best place to visit in Paris is a matter of personal preference, but some of the most popular and iconic attractions include the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, the Arc de Triomphe, and the Champs-Élysées.'

In [7]:
# wait! it does not remember to use hitsoty it just keep them

In [18]:
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder

# Prepare the input
inputs = {"input": "What is the best place there to visit?", "history": memory.chat_memory.messages}

# Format prompt manually
messages = prompt.format_messages(**inputs)

for m in messages:
    print(f"{m.type}: {m.content}")


system: You are a helpful assistant. first look at the history of chat messages and then answer based on that.
human: What's the capital of France?
ai: I'd be happy to help! According to our chat history, this is the first message you've sent. Therefore, I'll provide the answer: The capital of France is Paris.
human: What's the population of that?
ai: According to our chat history, you've asked about the capital of France, which is Paris. As for the population, Paris has a population of approximately 2.1 million people within its city limits. However, the larger metropolitan area of Paris has a population of over 12.2 million people.
human: the most famouse person ever lived there?
ai: According to our chat history, you've asked about the capital of France, which is Paris, and also about the population of Paris. As for your latest question, one of the most famous people to have lived in Paris is Vincent van Gogh, a Dutch post-impressionist artist who spent time in the city and created 

In [3]:
# ConversationSummaryMemory works as below:

from langchain.memory import ConversationSummaryMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create summary memory
summary_memory = ConversationSummaryMemory(
    llm=llm,
    return_messages=True
)

# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])

# Chain with memory
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=summary_memory
)

from langchain.schema import HumanMessage, AIMessage
summary_memory.chat_memory.add_user_message("What is the best place there to visit in Paris?")
summary_memory.chat_memory.add_ai_message("House of Meoosiso")

# Interact
chain.run("The best place to visit is House of Meoosiso.")
response = chain.run("What is the best place to visit?")
print(response)


/var/folders/1k/dtgfp49j1f92ysb1xwlxsnsm0000gp/T/ipykernel_42068/653734456.py:32: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("The best place to visit is House of Meoosiso.")


The best place to visit is House of Meoosiso.

I'm not familiar with a place called "House of Meoosiso". It's possible that it's a small, local attraction or a private residence. Can you please provide more information about House of Meoosiso, such as its location or what it's known for? This will help me better assist you in planning your trip.

If you're looking for recommendations on popular tourist destinations, I'd be happy to help you with that as well. What type of trip are you looking for (beach relaxation, city exploration, outdoor adventure, etc.)?


In [9]:
# look at the answer above. Is it really ideal?